# <center>Деплой модели</center>

Задача развёртывания приложения на сервере называется **деплой** (от англ. deployment — внедрение, развёртывание). По сути, это процесс трансформации исходного кода вашего приложения в рабочее состояние на конкретном сервере.

Для того чтобы внедрить модель в продакшн (в минимальном варианте), необходимо:

1) Сохранить обученную модель в файл.
2) Поднять сервер.
3) Доставить и запустить на нём свою модель.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/9cd2438d2cf4036896b4d84c00c5e62a/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_1_3.png)

Опыт подсказывает, что если модель создаётся «в вакууме», то в итоге её просто нельзя будет вывести в продакшн.

Дополнительной проблемой является то, что большинство библиотек для машинного обучения сфокусированы именно на **обучении**, а не на **предсказании**.

# <center>Сохранение и загрузка моделей: pickle и joblib</center>

## <center>Сериализация и десириализация</center>

Чтобы гарантировать сохранение всей структуры данных и получить её при загрузке обратно, используется сериализация.

**Сериализация** — это процесс трансформации любой структуры данных, поддерживаемой в языке, в последовательность битов (или байтов). Обратной операцией является **десериализации**.

In [10]:
# Пример десериализации
line = 'word1, word2, word3'
line.split(",")

['word1', ' word2', ' word3']

In [11]:
# Пример сериализации
",".join(['word1', 'word2', 'word3'])

'word1,word2,word3'

Это простейший пример, но на практике всё гораздо сложнее. Сериализовывать можно не только в текст (как с *CSV*, *JSON* и подобными форматами), но и в бинарный формат, который человек не сможет прочитать.

Бывают форматы, которые могут описывать более сложные структуры (тот же *JSON*). Также можно добавить сжатие итогового набора битов.

> Заметим, что программа должна потратить некоторый ресурс *CPU*, чтобы преобразовать объект в набор байтов и наоборот.

## <center>Инструменты сериализации: Pickle</center>

В стандартную библиотеку *Python* входит модуль `pickle`, который служит для сериализации почти всех объектов произвольного типа.

Мы помним, что объекты находятся в оперативной памяти и направляются в байтовые потоки ввода-вывода. В байтовые потоки может быть направлен любой файлоподобный объект.

**ШАГ №1**

Обучим модель линейной регрессии на встроенном датасете о диабете — *Diabetes dataset*.

В качестве модели, прогнозирующей целевую переменную, возьмём простейшую линейную регрессию, и обучим её на исходных данных:

In [12]:
from sklearn.linear_model import LinearRegression

from sklearn.datasets import load_diabetes

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression()
# Обучаем модель
regressor.fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

**ШАГ №2**

Когда мы получили обученную модель, нам необходимо сериализовать её, превратив объект *Python* в поток байтов. Для этого импортируем модуль `pickle` и воспользуемся функцией `dumps()`, в которую нужно передать объект *Python*.

In [13]:
import pickle

# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))

## bytes
## sklearn.linear_model._base.LinearRegression

<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


**ШАГ №3**

Попробуем восстановить (десериализовать) объект *Python*. Для этого в модуле `pickle` есть функция `loads()`, в которую нужно передать сериализованный объект (поток байтов).

In [14]:
# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

**ШАГ №4**

Сохраним сериализованный объект прямо в файл. Для этого в `pickle` есть функция `dump()` (без *s* на конце). В неё необходимо передать имя файла или ссылку на открытый файл. Файл назовём *myfile*, его расширение — `.pkl` (формат данных *pickle*):

In [15]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('data/myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

**ШАГ №5**

Посмотрим на код, который восстанавливает (десериализует) обученную модель из файла *myfile.pkl*. Для этого в *pickle* есть функция `load()` (без *s* на конце). В неё необходимо передать имя файла или ссылку на открытый файл.

In [16]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('data/myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file
## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression()

**ШАГ №6**

Убедимся, что методы и результаты предсказаний обученной модели и модели, загруженной из файла, совпадают:

In [17]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
all(regressor.predict(X) == regressor_from_bytes.predict(X))
all(regressor.predict(X) == regressor_from_file.predict(X))
## True

True

Как видите, исходная и восстановленная из байтов и файла модели дают одинаковые предсказания. Это значит, что теперь мы можем импортировать наши обученные модели в любое *Python*-приложение и пользоваться ими, минуя этап обучения и все этапы, предшествующие ему.

## <center>Ограничения</center>

Как мы упоминали, у *pickle* есть ограничения. Например, мы не можем сериализовать *лямбда*-функции. Давайте посмотрим, что нам вернёт следующий код:

In [18]:
# my_lambda = lambda x: x*2
# with open('data/my_lambda.pkl', 'wb') as output:
#     pickle.dump(my_lambda, output)
 
##"PicklingError: Can't pickle <function <lambda>"

> **Совет**. В таких случаях лучше пользоваться пакетом `dill`.

## <center>Сохранение пайплайна</center>

*Pickle* работает с любыми объектами *Python*. Поэтому для сохранения может быть доступна не просто обученная модель, но и целый **пайплайн**, включающий предобработку данных.

Например, мы хотим сериализовать пайплайн, который включает в себя min-max-нормализацию и отбор пяти наиболее важных факторов на основе корреляции Пирсона. Полученные в результате данные отправляются на вход модели линейной регрессии.

In [19]:
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)

# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000001E4A8DBE0C0>)),
                ('Linear', LinearRegression())])

In [20]:
# Сериализуем pipeline и записываем результат в файл
with open('data/my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [21]:
# Десериализуем pipeline из файла
with open('data/my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [22]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

## True

True


> Если мы хотим сохранять сериализованные пайплайны в виде потока байтов, нужно использовать функции `dumps()` и `loads()`, а не `dump()` и `load()`.

Однако в процессе предобработки могут возникнуть шаги, которые нельзя реализовать стандартными методами *sklearn*. Например, для решения многих задач в нашем курсе мы часто использовали *feature engineering*, чтобы повысить качество работы моделей. Для этого в *sklearn* можно организовать так называемые кастомные трансформеры. Такой трансформер должен наследоваться от двух классов: `TransformerMixin` и `BaseEstimator`.

У трансформера должно быть **три обязательных метода**:

* **`__init__()`** — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров. Например, у трансформера для создания полиномиальных признаков `PolynomialFeatures` из *sklearn* в методе `__init__()` параметр `degree` задаёт степень полинома.

* **`fit()`** — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (*self*).
Например, в трансформере `StandardScaler` в методе `fit()` прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода `fit()`.

* **`transform()`** — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода `transform()` у `StandardScaler` из *sklearn* внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе `fit()`.

Посмотрим на шаблон кастомного трансформера:

In [23]:
from sklearn.base import TransformerMixin, BaseEstimator
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X

Наш трансформер пока что ничего не делает. Предположим, мы хотим генерировать в данных новый признак, который является простым произведением первых трёх столбцов таблицы. Давайте пропишем в методе `transform()` эти действия.

Для работы такого трансформера нужны только исходные данные без дополнительных параметров, поэтому методы `__init__()` и `fit()` остаются без изменений.

In [24]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

In [25]:
import numpy as np

# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

## Shape before transform: (442, 10)
## Shape after transform: (442, 11)

Shape before transform: (442, 10)
Shape after transform: (442, 11)


Видно, что в результате трансформации в исходную матрицу наблюдений добавился новый столбец.

Теперь встроим этот трансформер в сам пайплайн — для этого достаточно добавить новый шаг в пайплайн.

In [26]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x000001E4A8DBE0C0>)),
                ('Linear', LinearRegression())])

In [27]:
# Сериализуем pipeline и записываем результат в файл
with open('data/my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [28]:
# Задание 2.5

# Десериализуем pipeline из файла
with open('data/my_new_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file) 
    
features =np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])
 
# Формируем предсказание
print(np.round(loaded_pipe.predict(features)))

[173.]


## <center>Библиотека Joblib</center>

Иногда массивы данных, на которых обучаются модели, бывают настолько большими, что после загрузки из *pickle* невозможно восстановить объект полностью.

В таких случаях вместо *pickle* лучше использовать библиотеку `joblib`. Этот модуль более эффективен и надёжен для работы с объектами, которые содержат большие массивы данных. Пожалуй, единственный минус этого модуля в том, что он может «консервировать» только в файл, поэтому вы не сможете получить объект в виде бинарной строки и работать с ним. В модуле попросту отсутствуют методы для работы с бинарной строкой. Формат файлов для сохранения — `.joblib`.

In [29]:
import joblib

# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Обучаем модель линейной регрессии
regressor = LinearRegression()
regressor.fit(X, y)
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'data/regr.joblib')

## ['regr.joblib']

['data/regr.joblib']

In [30]:
# Десериализуем модель из файла
clf_from_jobliv = joblib.load('data/regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

## True

True

In [31]:
# Задание 3.1

import pickle
 
with open('data/model.pkl', 'rb') as pkl_file:
    model = pickle.load(pkl_file)

secret word: skillfactory
how is this possible? answer is here: https://youtu.be/xm-A-h9QkXg


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [32]:
# Задание 3.2

model

LinearRegression(positive=True)

In [33]:
# Задание 3.3

print(model.predict([[1, 1, 1, 0.661212487096872]]))

[0.666]


In [34]:
# Задание 3.4

my_dict = {'a': model.a, 'b': model.b}

with open('data/my_dict.pkl', 'wb') as pkl_file:
    pickle.dump(my_dict, pkl_file)

!python data/hw1_check_ol.py data/my_dict.pkl

('secret code 2:', '3c508')


# <center>Сохранение и загрузка моделей: PMML и ONNX-ML</center>

Среда или требования к инференсу модели для проекта могут быть устроены так, что потребуют реализации на языке программирования, отличном от *Python*. Например, если компания разрабатывает десктопное приложение, то для внедрения модели её потребуется «перевести» на *Java* или *C++*.

## <center>Predictive Model Markup Language</center>

**PMML** — это *XML*-диалект, который применяется для описания статистических и *DS*-моделей. *PMML*-совместимые приложения позволяют легко обмениваться моделями данных между собой. Разработка и внедрение *PMML* осуществляется *IT*-консорциумом *Data Mining Group*.

К сожалению, далеко не все библиотеки для машинного обучения (в том числе `sklearn`) поддерживают возможность сохранения обученной модели в указанном формате. Однако для этого можно использовать сторонние библиотеки, и одной из самых популярных является `Nyoka`.

Сохраним модель из предыдущего блока в формат *PMML*.

Для установки можно использовать систему управления пакетами `pip`:

```python
pip install nyoka
```

In [35]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="data/pipeline.pmml")

Откроем файл *pipe.pmml* с помощью любого текстового редактора. Рассмотрим этот файл подробнее:

* Секция `<DataDictionary>` содержит информацию о признаках, включая наименование и тип данных, используемых для построения модели.

* Секция `<TransformationDictionary>` содержит информацию о необходимых преобразованиях для каждого признака. Обратите внимание, что в этом блоке также содержится информация для трансформации. Так как мы использовали `minMaxScaler()`, то в файле записаны минимальное и максимальное значения.

## <center>Open Neural Network Exchange</center>

**ONNX (Open Neural Network Exchange)** — это открытый стандарт для обеспечения совместимости моделей машинного обучения. Он позволяет разработчикам искусственного интеллекта использовать модели с различными инфраструктурами, инструментами, средами исполнения и компиляторами.

Для конвертации различных фреймворков (не только *DL*) в формат *ONNX* и обратно существует ряд библиотек:

* `ONNX-Tensorflow`;
* `Tensorflow-ONNX`;
* `Keras-ONNX`;
* `Sklearn-ONNX`.
* и другие.

Также в рамках стандарта *ONNX* есть инструмент *ONNX-runtime*. Он служит для ускорения инференса *Python*-моделей, а также инференса на других языках, например *Java*, *C++*.

В задаче ниже мы обучаем модель `sklearn`, конвертируем ее в *ONNX* и делаем инференс через *ONNX-runtime*.

```python
import onnxruntime as rt 
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(X_test)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в ONNX-формат
initial_type = [('float_input', FloatTensorType([None, 13]))]
model_onnx = convert_sklearn(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через ONNX-runtime
sess = rt.InferenceSession("model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx)
```

# <center>Деплой модели</center>

# <center>Протоколы сетевого взаимодействия</center>

* Во-первых, нужно разобраться с тем, как происходит взаимодействие серверов по сети.

* Во-вторых, необходимо узнать, как написать сервер и обернуть в него модель. Какие есть фреймворки? Какой выбрать конкретно в нашем случае? Как его реализовать?

## <center>Модели сетевого взаимодействия</center>

Наиболее известные модели сетевого взаимодействия — *OSI* и *TCP/IP*.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/a5479e1a2ef23577a9305345f9923344/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_5_1.png)

Эти модели распределяют сетевые протоколы по разным уровням взаимодействия. Вообще **протокол** — это некоторый набор правил, определяющий принципы взаимодействия устройств в сети. В нашем случае это правила, по которым программа, получив по сети набор битов, понимает, как его прочитать и что он значит.

Для того чтобы обмен информацией между устройствами проходил успешно, все устройства (участники процесса) должны следовать условиям протокола. В сети поддержка протоколов встраивается или в аппаратную (в «железо»), или в программную часть (в код системы), или в обе этих части. 

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/776616115a1b970957ee7bdcef380176/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_5_2.png)

В процессе сетевого взаимодействия участвуют как минимум два устройства — устройство-отправитель и устройство-получатель. Говоря простым языком, каждая из моделей сетевых взаимодействий устанавливает правила и регламенты по отправке сообщений между компьютерами.

Отправленное сообщение проходит все уровни, начиная от прикладного уровня приложений и заканчивая физическим уровнем доступа к сети. Когда сообщение доходит до адресата, оно также проходит все уровни в обратном порядке.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/09d73a8db432111eb6796ec41976b579/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_5_3.png)

Для наших целей (деплой модели в прод) достаточно уметь работать всего с тремя протоколами:

* **IP** — протокол сетевого уровня. Он определяет путь, по которому передаются данные.

* **TCP** — соответствует транспортному уровню, а значит, определяет, как передаются данные.

* **HTTP** — относится к прикладному уровню, описывающему взаимодействие приложений с сетью.

**IP**

**IP (Internet Protocol)** — один из главных протоколов сетевого взаимодействия. Он отвечает за маршрутизацию трафика по сети, то есть определяет путь, по которому отправятся данные. Данные передаются пакетами (или *датаграммами*), которые формирует протокол *IP*.

Важным свойством *IP* является отсутствие гарантированной доставки пакетов и их цельности: пакеты могут прийти в другой очерёдности (не в той, в которой их отправляли), прийти повреждёнными (тогда они уничтожаются) или вообще не прийти. Путь, по которому отправятся данные, строится на основе *IP-адресов*. *IP-адрес* — это уникальный адрес, используемый для связи устройств внутри сети.

|IP-адреса|Где используются|
|---------|----------------|
|127.0.0.1–127.255.255.255|используются для связи внутри локальной машины (localhost)|
|172.16.0.1–172.31.255.255|используются для частных подсетей (недоступных из интернета)|
|198.18.0.1–198.19.255.254|используются для тестирования производительности|

> **localhost** — зарезервированное доменное имя для IP-адресов из диапазона 127.0.0.1–127.255.255.255 (в сети из одного компьютера — для 127.0.0.1).

В компьютерной сети *localhost* относится к компьютеру, на котором запущена программа. Компьютер работает как виртуальный сервер. Тем самым создаётся так называемая «внутренняя петля»: обращаясь по *IP-адресу localhost*, вы, по сути, заставляете компьютер общаться с самим собой (хотя на самом деле внутри всё немного сложнее). Это нужно, например, для разработки и тестирования клиент-серверных приложений на одной машине (то есть и клиент, и сервер находятся на одном компьютере), что позволяет при разработке не использовать сетевое оборудование, дополнительные программные модули и тому подобное.

**TCP**

**TCP (Transmission Control Protocol)** — протокол транспортного уровня. Он отвечает за управление передачей данных и гарантирует:

* доставку пакетов (посылает пакеты повторно, если они не были доставлены);
* последовательность и целостность доставки пакетов (используя нумерацию и контрольные суммы для проверки);
* устраняет дубликаты в случае необходимости.

Важной особенностью *TCP* является то, что перед отправкой данных он «устанавливает соединение» с получателем — обменивается управляющей информацией. После отправки пакетов источник ждёт подтверждения от получателя, что пакеты были доставлены.

Обычно на одном узле сети (сервере, компьютере) работают несколько приложений/процессов одновременно. Для идентификации приложения на источнике и получателе используется порт, который задаётся целым неотрицательным числом. Процесс или приложение могут зарезервировать у ОС определённый порт, например, для передачи данных по сети.

Порты разделяют на **системные** (0–1023), и **пользовательские** (1024–49151). Некоторые номера портов определены для конкретных приложений, например:

* 22 — протокол *SSH* для безопасной передачи данных;
* 25 — протокол *SMTP* для незащищённой передачи e-mail-сообщений;
* 80 — протокол *HTTP*.


> Например, если приложение доступно по адресу 172.16.0.11:8001, это значит следующее:
> * 172.16.0.11 — *IP*-адрес;
> * 8001 — *TCP*-порт, отведённый приложению.

**HTTP**

**HTTP** — это наиболее широко используемый протокол. Все сайты, на которые вы заходите, работают по этому протоколу. Он был разработан именно для передачи содержимого *HTML*-страниц в интернете, но впоследствии стал использоваться и для других целей. Например, *HTTP* применяется для налаживания взаимодействия между сервисами в сложных системах. Этим он нам и интересен. Итак, *HTTP* — это протокол, который работает по принципу клиент-сервер.

Это означает, что во взаимодействии участвует **две программы**, причём в разных ролях. Одна из них — **клиент**, или «заказчик услуг», формирует запрос и отправляет его к серверу. **Сервер**, или «поставщик услуг», получив запрос, обрабатывает его, формирует ответ и возвращает его клиенту.

## <center>Структура HTTP-запросов</center>

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/446e3edf28d1be2b305ca5dc7aa61dce/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_5_6.png)

Запрос и ответ в *HTTP* являются строками, составленными в соответствии с протоколом.

**Запрос состоит** из трёх частей:

1) **Стартовая строка**, или *Request Line* — по ней определяется вид запроса.

2) **Заголовки запроса**, или *Request Headers* — дополнительные параметры запроса, в которых обычно передаётся служебная информация, например, в каком формате ожидается ответ или информация о клиенте.

3) **Тело запроса**, или *Request Message Body* — содержит данные для передачи. Эта часть присутствует не всегда.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c607e041f2821e75cf7acd4c3a911e92/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_5_7.png)

Первое, что указано в стартовой строке — это метод, или тип запроса. Есть набор стандартных методов:
* `GET` — обычно означает получение содержимого ресурса и не содержит тела.
* `POST` — наоборот, передача данных ресурсу.
* `PUT` — обновление ресурса.
* `DELETE` — удаление ресурса.

Адрес ресурса, или *URI (Uniform Resource Identifier)* — это то, что вы видите в адресной строке браузера. Он следует за методом в стартовой строке запроса.

Расшифруем аббревиатуры:

* *URI* — Uniform Resource Identifier (унифицированный идентификатор ресурса);
* *URL* — Uniform Resource Locator (унифицированный определитель местонахождения ресурса);
* *URN* — Unifrorm Resource Name (унифицированное имя ресурса).

Большинство считает, что **http://google.com** или **https://skillfactory.ru/** — это просто *URL*-адреса. Тем не менее, мы можем говорить о них как о *URI*. *URI* представляет собой комбинацию *URL*-адресов и *URN*. Таким образом, мы можем с уверенностью сказать, что все URL являются URI. Однако обратное неверно.

**Ответ** также состоит из стартовой строки, заголовков и тела.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/4c08c3a83c28594f597c19a65352d664/asset-v1:Skillfactory+DSMED+2023+type@asset+block/DSPROD_md1_5_11.png)

>Основное отличие — в стартовой строке: там вместо метода и *URI* указывается код состояния. Это численное значение, которое показывает результат обработки.

**Группы кодов** состояния ответа *HTTP*-сервера делятся на следующие группы:

* информационные (100–199);
* успешно (200–299);
* перенаправление (300–399);
* ошибка клиента (400–499);
* ошибка сервера (500–599).

## <center>REST (Representational State Transfer)</center>

Cпецификация *HTTP* не обязывает сервер понимать все методы, а также не указывает серверу, что он должен делать при получении запроса с тем или иным методом. Поэтому был изобретён архитектурный стиль *REST*.

Он даёт более верхнеуровневые указания, чем *HTTP*-протокол, а именно:

* как правильно организовывать адресацию к ресурсам;
* какие методы у этих ресурсов должны быть;
* какой ожидается результат.

Каждая единица информации (ресурс) однозначно определяется *URL*.

* GET-запрос `/rest/users` — получение информации обо всех пользователях.
* GET-запрос `/rest/users/125` — получение информации о пользователе с id=125.
* POST-запрос `/rest/users` — добавление нового пользователя.
* PUT-запрос `/rest/users/125` — изменение информации о пользователе с id=125.
* DELETE-запрос `/rest/users/125` — удаление пользователя с id=125.

# <center>Пишем сервер на Flask</center>

## <center>Инициализация веб-приложения</center>

Сначала необходимо установить `Flask`. Сделать это можно с помощью *pip*:

```python
pip install flask
```

Теперь импортируем его и создадим объект *Flask*-приложения.

```python
from flask import Flask
app = Flask(__name__)
```

Мы передаём `__name__` при инициализации класса `Flask`, чтобы определить имя, с которым будет использоваться этот модуль. `Flask` использует расположение файла как точку, к которой он привязывает ресурсы.

Теперь мы можем написать функцию, которая будет обрабатывать запросы, и прикрепить её к какому-то пути (*URI*). Это делается с помощью специального декоратора `route`.

```python
@app.route('/hello')
def hello_func():
    return 'hello!'
```

Наша функция пока не делает никакой обработки и просто отвечает строкой с приветствием. Нам осталось запустить приложение. Для этого выполним метод run, не забыв занести его в стандартный `main`.

```python
if __name__ == '__main__':

    app.run('localhost', 5000)
```

> В блоке `if __name__ == '__main__'` прописывается код, который не должен выполняться при импорте модуля. Переменная `__name__` — это специальная переменная, которая будет равна "`__main__`", только если файл запускается как основная программа, и выставляется равной имени модуля при импорте модуля.

Мы запускаем наш сервис в строке `app.run('localhost', 5000)`, указывая адрес сетевого интерфейса и порт, на котором будет работать сервер. В нашем случае мы работаем на локальной машине по *IP*-адресу 127.0.0.1, или *localhost* (то есть доступ к сервису может быть получен только с нашего компьютера), а номер порта, по которому можно отправлять запросы, — 5000.

Вы должны увидеть примерно такой текст:
```python
* Serving Flask app "server" (lazy loading)

* Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
* Debug mode: off
* Running on http://localhost:5000/ (Press CTRL+C to quit)
```

Теперь откройте браузер и зайдите по адресу http://localhost:5000/hello.

Вы увидите текст hello!, а в логах — строку с кодом обработки вашего запроса.

> Чтобы прервать работу программы-сервера в *VS Code*, нажмите сочетание клавиш *CTRL + C*, находясь в поле терминала. После этого перезапустите скрипт.

```python

# Задание 7.3

@app.route('/')
def index():
    return "Test message. The server is running"
```

## <center>Передача параметров запроса</center>

Параметры можно передавать тремя способами:

1) через адресную строку;
2) через заголовки;
3) через тело.

> В заголовках обычно передают сервисные параметры. Так как у метода `GET` не бывает тела, остаются только параметры адресной строки.

Если поставить в конце адреса вопросительный знак (**?**), то после него можно добавлять параметры. Передавать можно любые параметры — они будут перечисляться через **&** и состоять из имени и значения (**id=10**).

Параметры запроса во *Flask* находятся в специальном объекте `request`, который нужно импортировать. Параметры адресной строки можно найти в поле `args` этого объекта, где `args` — это словарь.

Давайте немного модифицируем наш код — теперь сервер будет здороваться и обращаться по имени. Для этого занесём параметр `name`, полученный из `request`, в переменную и воспользуемся этим значением, поставив его в форматированную строку.

```python
from flask import Flask, request

app = Flask(__name__)
@app.route('/hello')

def hello_func():
    name = request.args.get('name')
    return f'hello {name}!'

if __name__ == '__main__':

    app.run('localhost', 5000)
```

Перезапустите сервер и зайдите по адресу http://localhost:5000/hello?name=world.

В результате выполнения запроса вы должны увидеть в браузере текст "hello world".

```python
# Задание 7.4

from flask import Flask
import datetime
app = Flask(__name__)

@app.route('/time')
def current_time():
    return {'time': datetime.datetime.now()}
if __name__ == '__main__':
    app.run('localhost', 5000)

# Для выполнения запроса используется ссылка http://localhost:5000/time.
```

## <center>POST-запросы</center>

Пока что мы написали обработчик *GET*-запроса.

Если мы хотим обрабатывать и другие методы, например *POST*-запросы, это необходимо указать в декораторе:

```python
@app.route('/add', methods=['POST'])
```

Что должна возвращать функция-обработчик? Она будет возвращать объект `Response`, в котором мы должны были бы задать все компоненты ответа: код обработки, заголовки и тело. К счастью, *Flask* (а точнее, объект `Response`) умеет и превращать в ответы другие объекты, и самостоятельно формировать ответ.

Например, строка

```python
return f'hello {name}!'
```
преобразуется в ответ с кодом 200 и телом, состоящим из этой строки:
```python
return f'hello {name}!', 200
```

> Вспомогательная функция `jsonify` поможет преобразовать обычный питоновский словарь в ответ в формате *JSON*, который очень часто используется для передачи структурированных данных.

Напишем обработчик *POST*-запроса, который будет читать тело запроса в *JSON*-формате и составлять ответ на основе его содержимого.

```python
from flask import Flask, request, jsonify
app = Flask(__name__)
```

Функция-обработчик будет иметь следующий вид:

```python
@app.route('/add', methods=['POST']) # Укажем, что функция обрабатывает метод POST
def add():
    # Параметры тела доступны в поле data. Но если тело — это JSON-строка, то можно использовать поле json
    num = request.json.get('num')
        # Напишем проверку и укажем код ошибки
        if num > 10:
        return 'too much', 400
    return jsonify({
        'result': num + 1
    })
```

Запускаем:

```python
if __name__ == '__main__':

    app.run('localhost', 5000)
```

К сожалению, браузеры не умеют писать *POST*-запросы самостоятельно. Это реализуется только через клиентские приложения, поэтому нам не хватит обычного браузера, чтобы проверить сервис.

Напишем простой сервис в соседнем скрипте, используя библиотеку `requests`, которая позволяет отправлять *HTTP*-запросы. Для этого создадим отдельный файл *client.py*, в котором опишем работу программы клиента.

Полный код нашего простейшего клиентского приложения будет выглядеть так:
```python
import requests

if __name__ == '__main__':
    # выполняем POST-запрос на сервер по эндпоинту add с параметром json
    r = requests.post('http://localhost:5000/add', json={'num': 5})
    # выводим статус запроса
    print(r.status_code)
    # реализуем обработку результата
    if r.status_code == 200:
        # если запрос выполнен успешно (код обработки=200),
        # выводим результат на экран
        print(r.json()['result'])
    else:
        # если запрос завершён с кодом, отличным от 200,
        # выводим содержимое ответа
        print(r.text)
```

> Чтобы запустить код клиентского приложения параллельно работающему серверу в *VS Code*, создайте новый терминал и запустите файл *client.py* вручную, написав в терминале команду: python client.py

Выполнив данный код, мы произведём *POST*-запрос на локальный сервер по эндпоинту `add`. Так как переданное значение `num < 10`, то запрос будет выполнен успешно и в терминале, соответствующему программе-клиенту, будет выведено:
```python
## 200
## 6
```

```python
# Задание 7.6

from flask import Flask, request, jsonify
import pickle
import numpy as np

with open('./models/model.pkl', 'rb') as pkl_file: 
    model = pickle.load(pkl_file)

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    features = np.array(request.json)
    features = features.reshape(1, 4)
    prediction = model.predict(features)
    return  jsonify({'prediction': prediction[0]})

if __name__ == '__main__':
    
    app.run('localhost', 5000)
```

# <center>GIL. uWSGI + NGINX</center>

> Инструменты, рассматриваемые в этом юните, предназначены исключительно для *UNIX*-операционных систем (*Linux* и *MacOS*).

Представим ситуацию: мы хотим, чтобы сервер с нашей моделью одновременно обрабатывал несколько процессов. Если мы попробуем запустить такой сервер, то либо его производительность будет крайне низкой, либо процессы просто не запустятся.

Дело в механизме под названием *GIL — Global Interpreter Lock*. Он заключается в том, что в любом процессе *Python* одновременно может работать только один тред (один конкретный поток).